Importo las librerías que voy a utilizar

In [165]:
import pandas as pd
import numpy as np
import fastparquet
import pyarrow
import matplotlib.pyplot as plt 
import seaborn as sns
pd.set_option('display.max_columns', 40)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_validate
from scipy import stats
from sklearn.ensemble import RandomForestClassifier

Traigo el dataset "train.parquet" utilizando las librerías pyarrow y fastparquet

In [166]:
df_train = pd.read_parquet('train.parquet') # Lee el dataset
df_train.head() # Imprime las 5 primeras filas del dataset

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,1,1,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,1,1,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,0,1,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,1,1,0,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,1,1,0,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn


Hago una visualización de los datos para tener una mejor comprensión del dataset. 

In [167]:
df_train.info() # Despliega todas las columnas y sus respectivos tipos de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346479 entries, 0 to 346478
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       346479 non-null  int64  
 1   url                      346479 non-null  object 
 2   region                   346479 non-null  object 
 3   region_url               346479 non-null  object 
 4   price                    346479 non-null  int64  
 5   type                     346479 non-null  object 
 6   sqfeet                   346479 non-null  int64  
 7   beds                     346479 non-null  int64  
 8   baths                    346479 non-null  float64
 9   cats_allowed             346479 non-null  int64  
 10  dogs_allowed             346479 non-null  int64  
 11  smoking_allowed          346479 non-null  int64  
 12  wheelchair_access        346479 non-null  int64  
 13  electric_vehicle_charge  346479 non-null  int64  
 14  come

In [168]:
df_train.isnull().sum() # Imprime la suma de los valores nulos de cada columna

id                              0
url                             0
region                          0
region_url                      0
price                           0
type                            0
sqfeet                          0
beds                            0
baths                           0
cats_allowed                    0
dogs_allowed                    0
smoking_allowed                 0
wheelchair_access               0
electric_vehicle_charge         0
comes_furnished                 0
laundry_options             71171
parking_options            126682
image_url                       0
description                     2
lat                          1722
long                         1722
state                           0
dtype: int64

Con la función get_cummies, convierto columnas categóricas en numéricas para poder analizarlas.

Creo la columna 'category_price', que va a almacenar los valores categóricos de la columna 'price', divididos en bajos, medios y altos para posteriormente poder entrenar el modelo.

In [169]:
def categorise(row):  # Le asigno valor 1 a todos los valores de la columna 'price' que estén entre 0 y 999 dólares, y el valor 0 a los que superen dicho monto.
    if row['price'] >= 0 and row['price'] <= 999:
        return 1
    else: return 0
df_train['category_price'] = df_train.apply(lambda row: categorise(row), axis=1) # La función lambda agrega las categorías creadas por la función "categorize" a la nueva columna.

Muestro el nuevo dataset con las columnas dummies y la nueva columna categórica.

In [170]:
df_train.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state,category_price
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,1,1,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id,0
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,1,1,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co,0
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,0,1,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va,0
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,1,1,0,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az,0
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,1,1,0,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn,1


Elimino algunas columnas.

In [171]:
df_train.drop(['id', 'url', 'region_url', 'image_url', 'description', 'lat', 'long','laundry_options', 'parking_options', 'region', 'type', 'state', 'price'],axis=1, inplace=True) 

In [172]:
df_train.head()

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,category_price
0,1200,2,2.0,1,1,1,0,0,0,0
1,694,1,1.0,1,1,1,0,0,0,0
2,900,2,2.0,0,0,1,0,0,0,0
3,1469,3,2.0,1,1,1,0,0,0,0
4,700,1,1.0,1,1,1,0,0,0,1


Separo el dataframe para poder entrenarlo

In [173]:
X = df_train.drop(columns=['category_price'])
y = df_train['category_price']

In [174]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

Entreno el modelo de cladificación

In [175]:
classifier = RandomForestClassifier(n_estimators = 100)
classifier.fit(X_train, y_train)

RandomForestClassifier()

Hago la predicción

In [176]:
y_pred = classifier.predict(X_test)

In [177]:
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[43263 12668]
 [10965 37048]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.77      0.79     55931
           1       0.75      0.77      0.76     48013

    accuracy                           0.77    103944
   macro avg       0.77      0.77      0.77    103944
weighted avg       0.77      0.77      0.77    103944

Accuracy: 0.7726371892557531


In [178]:
X

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished
0,1200,2,2.0,1,1,1,0,0,0
1,694,1,1.0,1,1,1,0,0,0
2,900,2,2.0,0,0,1,0,0,0
3,1469,3,2.0,1,1,1,0,0,0
4,700,1,1.0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
346474,660,1,1.0,1,1,1,0,0,0
346475,1099,2,2.0,1,1,1,0,0,0
346476,1104,2,2.0,1,1,0,0,0,0
346477,1050,2,2.0,0,0,0,0,0,0


Traigo el dataset "test.parquet"

In [179]:
df_test = pd.read_parquet('test.parquet')

In [180]:
df_test.drop(['id', 'url', 'region_url', 'image_url', 'description', 'lat', 'long','laundry_options', 'parking_options', 'region', 'type', 'state'],axis=1, inplace=True)

In [181]:
df_test

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished
0,1344,3,2.0,0,0,1,0,0,0
1,1050,2,1.0,0,0,1,0,0,0
2,1150,2,2.0,1,1,1,1,0,0
3,1280,2,2.5,1,1,0,0,0,0
4,783,2,1.0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
38493,1180,2,2.0,1,1,0,1,0,0
38494,1138,3,2.0,1,1,1,0,0,0
38495,743,1,1.0,1,1,1,0,0,0
38496,1276,3,2.0,0,0,0,0,0,0


In [194]:
y_prueba = np.random.choice(y_pred, 38498)
y_prueba

array([0, 0, 1, ..., 1, 0, 1])

In [195]:
X = df_test
y = y_prueba

In [196]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [197]:
classifier = RandomForestClassifier(n_estimators = 100)
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [198]:
y_pred = classifier.predict(X_test)

In [199]:
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[5350 1796]
 [3244 1160]]
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.75      0.68      7146
           1       0.39      0.26      0.32      4404

    accuracy                           0.56     11550
   macro avg       0.51      0.51      0.50     11550
weighted avg       0.53      0.56      0.54     11550

Accuracy: 0.5636363636363636
